In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [2]:
import findspark
findspark.init('../spark-3.2.0-bin-hadoop3.2')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('ML-logistic-regression').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/05 07:51:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.ml.classification import LogisticRegression
# Load training data
data = spark.read.format("libsvm").load("../data/sample_libsvm_data.txt")

24/09/05 07:51:14 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [6]:
data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [7]:
data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
+-----+--------------------+
only showing top 10 rows



In [8]:
model = LogisticRegression()

In [9]:
train_data,test_data = data.randomSplit([0.7,0.3])

In [10]:
final_model = model.fit(train_data)

24/09/05 07:53:09 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/09/05 07:53:09 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
24/09/05 07:53:09 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/09/05 07:53:09 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [13]:
predictions_and_labels = final_model.evaluate(test_data)

In [15]:
predictions_and_labels.predictions.show()

/home/ubuntu/ML-Learning/../spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[19.862516747642,...|[0.99999999763506...|       0.0|
|  0.0|(692,[98,99,100,1...|[30.4377776253502...|[0.99999999999993...|       0.0|
|  0.0|(692,[124,125,126...|[35.4401644305613...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[25.6517164135632...|[0.99999999999276...|       0.0|
|  0.0|(692,[125,126,127...|[32.3141985680754...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[24.9346847690266...|[0.99999999998517...|       0.0|
|  0.0|(692,[126,127,128...|[28.9596846520251...|[0.99999999999973...|       0.0|
|  0.0|(692,[127,128,129...|[25.0318302652131...|[0.99999999998654...|       0.0|
|  0.0|(692,[152,153,154...|[17.5325904472509...|[0.99999997569516...|       0.0|
|  0.0|(692,[154

In [22]:
bcEvaluator = BinaryClassificationEvaluator()

In [23]:
roc = bcEvaluator.evaluate(predictions_and_labels.predictions)

In [24]:
roc

1.0